# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

Импортируем библиотеки

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

!pip3 install catboost
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier, Pool

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.dummy import DummyClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

Сохраним данные в переменную df_churn и посмотрим на них

In [2]:
try:
    df_churn = pd.read_csv(r'Churn.csv')
    df_churn.head()
except:
    df_churn = pd.read_csv('/datasets/Churn.csv')
    df_churn.head()

Посмотрим информацию о данных

In [3]:
df_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


In [4]:
df_churn.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


Приведем названия столбцов к нижнему регистру

In [5]:
df_churn.columns = df_churn.columns.str.lower()
df_churn.columns

Index(['rownumber', 'customerid', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard',
       'isactivemember', 'estimatedsalary', 'exited'],
      dtype='object')

In [6]:
df_churn.rename(
    {'rownumber': 'row_number',
     'customerid': 'customer_id',
     'creditscore': 'credit_score',
     'numofproducts': 'num_of_products',
     'hascrcard': 'has_cr_card',
     'isactivemember': 'is_active_member',
     'estimatedsalary': 'estimated_salary'},
    axis=1, inplace=True)

Так же, у нас есть пропуски в колонке с кол-вом лет в банке, возможно это новые клиенты банка.

Посмотрим на пропущенные данные и на значения с нулем

In [7]:
df_churn[df_churn['tenure'] == 0]

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
29,30,15656300,Lucciano,411,France,Male,29,0.0,59697.17,2,1,1,53483.21,0
35,36,15794171,Lombardo,475,France,Female,45,0.0,134264.04,1,1,0,27822.99,1
57,58,15647091,Endrizzi,725,Germany,Male,19,0.0,75888.20,1,0,0,45613.75,0
72,73,15812518,Palermo,657,Spain,Female,37,0.0,163607.18,1,0,1,44203.55,0
127,128,15782688,Piccio,625,Germany,Male,56,0.0,148507.24,1,1,0,46824.08,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9793,9794,15772363,Hilton,772,Germany,Female,42,0.0,101979.16,1,1,0,90928.48,0
9799,9800,15722731,Manna,653,France,Male,46,0.0,119556.10,1,1,0,78250.13,1
9843,9844,15778304,Fan,646,Germany,Male,24,0.0,92398.08,1,1,1,18897.29,0
9868,9869,15587640,Rowntree,718,France,Female,43,0.0,93143.39,1,1,0,167554.86,0


In [8]:
df_churn[df_churn['tenure'].isna()]

,row_number,customer_id,surname,credit_score,geography,gender,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited
30,31,15589475,Azikiwe,591,Spain,Female,39,NaN,0.00,3,1,0,140469.38,1
48,49,15766205,Yin,550,Germany,Male,38,NaN,103391.38,1,0,1,90878.13,0
51,52,15768193,Trevisani,585,Germany,Male,36,NaN,146050.97,2,0,0,86424.57,0
53,54,15702298,Parkhill,655,Germany,Male,41,NaN,125561.97,1,0,0,164040.94,1
60,61,15651280,Hunter,742,Germany,Male,35,NaN,136857.00,1,0,0,84509.57,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9944,9945,15703923,Cameron,744,Germany,Male,41,NaN,190409.34,2,1,1,138361.48,0
9956,9957,15707861,Nucci,520,France,Female,46,NaN,85216.61,1,1,0,117369.52,1
9964,9965,15642785,Douglas,479,France,Male,34,NaN,117593.48,2,0,0,113308.29,0
9985,9986,15586914,Nepean,659,France,Male,36,NaN,123841.49,2,1,0,96833.00,0


Никакой связи между данными не прослеживается, так что избавимся от них

In [9]:
df_churn = df_churn.dropna()
df_churn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9091 entries, 0 to 9998
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   row_number        9091 non-null   int64  
 1   customer_id       9091 non-null   int64  
 2   surname           9091 non-null   object 
 3   credit_score      9091 non-null   int64  
 4   geography         9091 non-null   object 
 5   gender            9091 non-null   object 
 6   age               9091 non-null   int64  
 7   tenure            9091 non-null   float64
 8   balance           9091 non-null   float64
 9   num_of_products   9091 non-null   int64  
 10  has_cr_card       9091 non-null   int64  
 11  is_active_member  9091 non-null   int64  
 12  estimated_salary  9091 non-null   float64
 13  exited            9091 non-null   int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.0+ MB


Заменим тип данных на цельночисленный

In [10]:
df_churn['tenure'] = df_churn['tenure'].astype(int)

В таблице присутствуют данные не несущие пользы для дальнейшего обучения моделей - это номер строки, id пользователя и фамилия

Избавимся от них

In [11]:
df_churn = df_churn.drop(['row_number', 'customer_id', 'surname'], axis=1)

Закодируем данные методом OHE

In [12]:
df_ohe = pd.get_dummies(data=df_churn, columns=["gender", "geography"], drop_first=True)

df_ohe.head()

,credit_score,age,tenure,balance,num_of_products,has_cr_card,is_active_member,estimated_salary,exited,gender_Male,geography_Germany,geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1


Разделим данные на выборки в соотношении 60-20-20

In [13]:
features = df_ohe.drop('exited', axis=1)
target = df_ohe['exited']

In [14]:
features_train, features_tv, target_train, target_tv = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)
features_test, features_valid, target_test, target_valid = train_test_split(
    features_tv, target_tv, test_size=0.5, random_state=12345, stratify=target_tv)

И стандартизируем количественные признаки

In [15]:
numeric = ['age', 'tenure', 'balance', 'estimated_salary', 'credit_score']
scaler = StandardScaler()
scaler.fit(features_train[numeric]) 

pd.options.mode.chained_assignment = None
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

### Вывод

Мы изучили данные и подготовили их к дальнейшей работе, подготовили признаки и целевые признаки, а так же стандартизировали данные. Теперь можно приступать к обучению моделей.

## Исследование задачи

Посмотрим на баланс классов в наших выборках

In [16]:
print('features_train :\n', target_train.value_counts(),
     '\nfeatures_valid :\n', target_valid.value_counts(),
     '\nfeatures_test :\n', target_test.value_counts()
     )

features_train :
 0    4342
1    1112
Name: exited, dtype: int64 
features_valid :
 0    1448
1     371
Name: exited, dtype: int64 
features_test :
 0    1447
1     371
Name: exited, dtype: int64


Мы видим что имеется сильный дисбаланс классов в сторону отричательного класса

Попробуем обучить модель на таких пропорциях

In [17]:
forest = RandomForestClassifier(random_state=12345, n_estimators=100)
forest.fit(features_train, target_train)
predicted_valid_forest = forest.predict(features_valid)

Напишем функцию для отображения метрик

In [18]:
def metrics(features, target, predict, model):
    print(confusion_matrix(target, predict))
    print('F1 =', f1_score(target, predict))
    probabilities_valid = model.predict_proba(features)
    probabilities_one_valid = probabilities_valid[:, 1]
    print('AUC-ROC =', roc_auc_score(target, probabilities_one_valid))

In [19]:
metrics(features_valid, target_valid, predicted_valid_forest, forest)

[[1384   64]
 [ 201  170]]
F1 = 0.5619834710743802
AUC-ROC = 0.8426056573989964


### Вывод

Мы видим что при дисбалансе классов наша модель довольно хорошо предсказывает отрицательные классы, но при этом плохо положительные из за меньшего колличества данных, попытаемся исправить эту проблему балансировкой классов. Так же попробуем разные модели и подберем для них гиперпараметры

## Борьба с дисбалансом

### Upsampling

Сбалансируем классы увеличив кол-во отрицательных

In [20]:
def upsample(features, target, repeat):
    
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled = shuffle(features_upsampled, random_state=12345)
    target_upsampled = shuffle(target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

    
repeat = round(target_train.value_counts().values[0] / target_train.value_counts().values[1])   
features_upsampled, target_upsampled = upsample(features_train, target_train, repeat)

print(features_upsampled.shape)
print(target_upsampled.shape)
print(target_upsampled.value_counts())

(8790, 11)
(8790,)
1    4448
0    4342
Name: exited, dtype: int64


Обучим первые модели и подберем для них гиперпараметры с помошью grid search

In [21]:
forest = RandomForestClassifier()

forest_params = {'n_estimators': [100, 150, 200, 250, 500],
                     'max_depth': [3, 4, 5],
                     'random_state': [42]
                }

gs_forest = GridSearchCV(forest, forest_params, cv=5, scoring='f1')
gs_forest.fit(features_upsampled, target_upsampled)
best_forest = gs_forest.best_estimator_       
predicted_valid_forest = best_forest.predict(features_valid) 
metrics(features_valid, target_valid, predicted_valid_forest, best_forest)

[[1197  251]
 [ 118  253]]
F1 = 0.5782857142857143
AUC-ROC = 0.8500841387321112


In [22]:
cat = CatBoostClassifier(logging_level='Silent')

grid = {'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 5, 9]}

grid_search_result = cat.grid_search(grid,
                                       X=features_upsampled,
                                       y=target_upsampled,
                                       verbose=False)

In [23]:
grid_search_result['params']

{'depth': 10, 'l2_leaf_reg': 1}

In [24]:
best_cat = CatBoostClassifier(verbose = False,
    depth = 10,
    iterations = 1000,
    l2_leaf_reg = 1,
    random_state = 42)
best_cat.fit(features_upsampled, target_upsampled)
predicted_valid_cat = best_cat.predict(features_valid)
metrics(features_valid, target_valid, predicted_valid_cat, best_cat)

[[1340  108]
 [ 167  204]]
F1 = 0.5973645680819911
AUC-ROC = 0.8368564876174592


In [25]:
log = LogisticRegression(solver='liblinear')
log_params = {
                   'intercept_scaling': [0.5, 1.0, 1.5],
                   'class_weight': [None, 'balanced'],
                   'C': [0.5, 1, 1.5],
                   'random_state': [42]
                   }

gs_log = GridSearchCV(log, log_params, cv=5, scoring='f1')
gs_log.fit(features_upsampled, target_upsampled)
best_log = gs_log.best_estimator_
predicted_valid_log = best_log.predict(features_valid)
metrics(features_valid, target_valid, predicted_valid_log, best_log)

[[1033  415]
 [ 106  265]]
F1 = 0.5042816365366317
AUC-ROC = 0.778186475257256


Мы видим что лучше всего себя показали catboost и случайный лес, а так же на них мы перешли порог f1 рейтинга на валидационной выборке. AUC ROC показатель так же на неплохом уровне, что означает улучшение соотшения TPR и FPR.

### Downsampling

Попробуем выровнять баланс уменьшением выборки

In [26]:
def downsample(features_train, target_train, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    features_downsampled = pd.concat(
    [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
    [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    features_downsampled, target_downsampled = shuffle(
    features_downsampled, target_downsampled, random_state=12345)
    return features_downsampled, target_downsampled

fraction = round(target_train.value_counts().values[1] / target_train.value_counts().values[0], 2)
features_downsampled, target_downsampled = downsample(features, target, fraction)

print(features_downsampled.shape)
print(target_downsampled.shape)
print(target_downsampled.value_counts())

(3736, 11)
(3736,)
0    1882
1    1854
Name: exited, dtype: int64


И так же обучим на них модели

In [27]:
forest_down = RandomForestClassifier()

forest_params = {'n_estimators': [100, 150, 200, 250, 500],
                     'max_depth': [3, 4, 5],
                     'random_state': [42]
                }

gs_forest = GridSearchCV(forest_down, forest_params, cv=5, scoring='f1')
gs_forest.fit(features_downsampled, target_downsampled)
best_forest_down = gs_forest.best_estimator_       
predicted_valid_forest = best_forest_down.predict(features_valid) 
metrics(features_valid, target_valid, predicted_valid_forest, best_forest_down)

[[1429   19]
 [ 322   49]]
F1 = 0.2232346241457859
AUC-ROC = 0.7527270628881177


In [28]:
cat_down = CatBoostClassifier(logging_level='Silent')

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 5, 9]}

grid_search_result = cat_down.grid_search(grid,
                                       X=features_downsampled,
                                       y=target_downsampled,
                                       verbose=False)

In [29]:
grid_search_result['params']

{'depth': 6, 'l2_leaf_reg': 5, 'learning_rate': 0.1}

In [30]:
best_cat_down = CatBoostClassifier(verbose = False,
    depth = 6,
    iterations = 1000,
    l2_leaf_reg = 1,
    learning_rate = 0.03,
    random_state = 42)
best_cat_down.fit(features_downsampled, target_downsampled)
predicted_valid_bestcat = best_cat_down.predict(features_valid)
metrics(features_valid, target_valid, predicted_valid_cat, best_cat_down)

[[1340  108]
 [ 167  204]]
F1 = 0.5973645680819911
AUC-ROC = 0.5619080505130228


In [31]:
log_down = LogisticRegression(solver='liblinear')
log_params = {
                   'intercept_scaling': [0.5, 1.0, 1.5],
                   'class_weight': [None, 'balanced'],
                   'C': [0.5, 1, 1.5],
                   'random_state': [42, 12345]
                   }

gs_log = GridSearchCV(log_down, log_params, cv=5, scoring='f1')
gs_log.fit(features_downsampled, target_downsampled)
best_log_down = gs_log.best_estimator_
predicted_valid_log = best_log_down.predict(features_valid)
metrics(features_valid, target_valid, predicted_valid_log, best_log_down)

[[1134  314]
 [ 157  214]]
F1 = 0.47608453837597325
AUC-ROC = 0.7319492635999464


При уменьшении выборки результаты ухудшились, исключением является catboost, но несмотря на хороший f1 рейтинг, просел AUC ROC. 
В остальном же результаты стали значительно хуже, из за уменьшения данных модели стали хуже находить закономерности.
Далее проверим наши модели на тестовой выборке и определим лучшую.

## Тестирование модели

Проверим модели на тестовой выборке

In [32]:
predicted_test_forest = best_forest.predict(features_test)
predicted_test_cat = best_cat.predict(features_test)
predicted_test_log = best_log.predict(features_test)
print('Метрики случайного леса')
print('_______________________')
metrics(features_test, target_test, predicted_test_forest, best_forest)
print('\nМетрики catboost')
print('_______________________')
metrics(features_test, target_test, predicted_test_cat, best_cat)
print('\nМетрики логистической регресии')
print('_______________________')
metrics(features_test, target_test, predicted_test_log, best_log)

Метрики случайного леса
_______________________
[[1168  279]
 [  94  277]]
F1 = 0.5976267529665589
AUC-ROC = 0.8748670453042544

Метрики catboost
_______________________
[[1345  102]
 [ 155  216]]
F1 = 0.6269956458635704
AUC-ROC = 0.8576681562559959

Метрики логистической регресии
_______________________
[[1010  437]
 [  98  273]]
F1 = 0.5050878815911193
AUC-ROC = 0.7907633043176979


Обучим модели на обьединеных тренировачных и валидационных выборках и проведем тестирование еще раз

In [33]:
# обьеденим тренировочную и валидационные выборки
features_samp = pd.concat([features_train] + [features_valid])
target_samp = pd.concat([target_train] + [target_valid])
features_samp = shuffle(features_samp, random_state=12345)
target_samp = shuffle(target_samp, random_state=12345)
# сбалансируем классы
repeat = round(target_samp.value_counts().values[0] / target_samp.value_counts().values[1])   
features_upsampled, target_upsampled = upsample(features_samp, target_samp, repeat)
# обучим модели на новых данных
best_forest.fit(features_upsampled, target_upsampled)
best_cat.fit(features_upsampled, target_upsampled)

In [34]:
predicted_test_forest = best_forest.predict(features_test)
predicted_test_cat = best_cat.predict(features_test)
print('Метрики случайного леса')
print('_______________________')
metrics(features_test, target_test, predicted_test_forest, best_forest)
print('\nМетрики catboost')
print('_______________________')
metrics(features_test, target_test, predicted_test_cat, best_cat)

Метрики случайного леса
_______________________
[[1161  286]
 [  82  289]]
F1 = 0.6109936575052854
AUC-ROC = 0.8766795135208638

Метрики catboost
_______________________
[[1325  122]
 [ 156  215]]
F1 = 0.6073446327683617
AUC-ROC = 0.8600934734379337


У нас лидируют случайный лес и catboost, а так же они закрывают порог f1 рейтинга в 0.59. Они так же показывают не плохие AUC ROC метрики.

##  Общий вывод

В исследовании на были предоставлены данные о поведении клиентов и расторжении договоров с банком. Мы предобработали их, распределили на признаки и целевой признак. Далее изучили как себя ведет модель при дисбалансе классов и избавились от него разными методами. В результате лучшие модели получились при увеличении выборки. Порок f1 рейтинга 0.59 прошли 2 модели - случайный лес и catboost. Catboost выглядит немного лучше из за его способности работать на малых выборках и является более универсальным.